In [1]:
!pip install pandas nltk spacy scikit-learn
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 67.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import pandas as pd

data = {
    "Intent": [
        "Order Status", "Order Status", "Order Status",
        "Return Policy", "Return Policy",
        "Product Info", "Product Info",
        "Delivery Info", "Delivery Info",
        "Greeting", "Goodbye", "Complaint"
    ],
    "Example Query": [
        "Where is my order #45672?",
        "Track my order #98765",
        "Has my package #33421 been shipped?",
        "How do I return an item?",
        "I want to cancel my return request",
        "Does this laptop have SSD storage?",
        "Is this phone waterproof?",
        "When will my order arrive?",
        "How long does delivery take?",
        "Hi there!",
        "Thanks, bye!",
        "I'm unhappy with my last order"
    ],
    "Response": [
        "Your order {order_id} is out for delivery.",
        "Your order {order_id} is being processed.",
        "Your package {order_id} has been shipped and will arrive soon.",
        "You can return items within 15 days using our returns portal.",
        "Your return request has been cancelled.",
        "Yes, this laptop includes SSD storage.",
        "Yes, this phone is water-resistant.",
        "Your order will arrive in 3–5 business days.",
        "Delivery usually takes 3–7 days depending on your location.",
        "Hello! How can I help you today?",
        "Goodbye! Have a nice day.",
        "Sorry to hear that. Please share your order ID so we can check."
    ]
}

df = pd.DataFrame(data)
df.to_csv("customer_support_dataset.csv", index=False)
df.head()


,Intent,Example Query,Response
0,Order Status,Where is my order #45672?,Your order {order_id} is out for delivery.
1,Order Status,Track my order #98765,Your order {order_id} is being processed.
2,Order Status,Has my package #33421 been shipped?,Your package {order_id} has been shipped and w...
3,Return Policy,How do I return an item?,You can return items within 15 days using our ...
4,Return Policy,I want to cancel my return request,Your return request has been cancelled.


In [14]:
!pip install vaderSentiment


In [15]:
import re
import nltk
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt_tab', quiet=True) # Download the missing resource

nlp = spacy.load("en_core_web_sm")
data = pd.read_csv("customer_support_dataset.csv")

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess(text):
    text = text.lower().translate(str.maketrans("", "", string.punctuation))
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

data["processed_query"] = data["Example Query"].apply(preprocess)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data["processed_query"])

def extract_order_id(text):
    match = re.search(r"#\d+", text)
    return match.group(0) if match else None

def classify_intent(query):
    query_processed = preprocess(query)
    query_vec = vectorizer.transform([query_processed])
    sim = cosine_similarity(query_vec, X)
    idx = sim.argmax()
    score = sim[0][idx]
    if score < 0.2:
        return None, None
    return data.iloc[idx]["Intent"], data.iloc[idx]["Response"]
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def detect_sentiment(text):
    score = analyzer.polarity_scores(text)["compound"]
    if score <= -0.3:
        return "negative"
    elif score >= 0.3:
        return "positive"
    else:
        return "neutral"
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def detect_sentiment(text):
    score = analyzer.polarity_scores(text)["compound"]
    if score <= -0.3:
        return "negative"
    elif score >= 0.3:
        return "positive"
    else:
        return "neutral"



def chatbot_response(query):
    sentiment = detect_sentiment(query)
    order_id = extract_order_id(query)
    intent, response = classify_intent(query)

    if not intent:
        if sentiment == "negative":
            return "I'm sorry to hear that. Let me connect you to a human support agent."
        return "I'm not sure about that. Please contact customer support."

    if "{order_id}" in response:
        response = response.format(order_id=order_id if order_id else "#00000")

    if sentiment == "negative":
        response += " I understand this might be frustrating. We’ll try to fix it quickly."

    return response


In [6]:
queries = [
    "Where is my order #23456?",
    "How do I return a product?",
    "Does this phone support fast charging?",
    "How long does delivery take?",
    "Hi!",
    "I’m upset with my order",
    "bye"
]

for q in queries:
    print("You:", q)
    print("Bot:", chatbot_response(q))
    print()


You: Where is my order #23456?
Bot: Your order #23456 is out for delivery.

You: How do I return a product?
Bot: You can return items within 15 days using our returns portal.

You: Does this phone support fast charging?
Bot: Yes, this phone is water-resistant.

You: How long does delivery take?
Bot: Delivery usually takes 3–7 days depending on your location.

You: Hi!
Bot: Hello! How can I help you today?

You: I’m upset with my order
Bot: Your order #00000 is out for delivery.

You: bye
Bot: Goodbye! Have a nice day.



In [7]:
!pip install gradio



In [8]:
import gradio as gr

def gradio_chatbot(query):
    return chatbot_response(query)

demo = gr.Interface(
    fn=gradio_chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask about your order, return, or delivery..."),
    outputs="text",
    title="🛍️ Customer Support Chatbot",
    description="Ask me about your orders, delivery, returns, or product information."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a6d2268dcdae99c88.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
